Code from : https://github.com/online-ml/river/blob/main/river/ensemble/ewa.py

In [ ]:
import collections
from typing import List

from river import base, linear_model, naive_bayes, tree

from river import datasets
from river import ensemble
from river import evaluate
from river import linear_model
from river import metrics
from river import naive_bayes
from river import preprocessing
from river import tree
from river import base

In [34]:
class VotingClassifier1(base.Classifier, base.Ensemble):
    """Voting classifier.
    """

    def __init__(self, models: List[base.Classifier], use_probabilities=True):
        super().__init__(models)
        self.use_probabilities = use_probabilities

    def learn_one(self, x, y):
        for model in self:
            model.learn_one(x, y)
        return self

    def predict_one(self, x):
        if self.use_probabilities:
            votes = (model.predict_proba_one(x) for model in self)
        else:
            votes = ({model.predict_one(x): 1} for model in self)
        agg = collections.Counter()
        for vote in votes:
            agg.update(vote)
        return agg.most_common(1)[0][0]

In [35]:
dataset = datasets.Bananas()

In [36]:
model_lr = (
     preprocessing.StandardScaler() |
         linear_model.LogisticRegression()
     )
 

model_ht = (
     preprocessing.StandardScaler() |
         tree.HoeffdingTreeClassifier()
     )
 

model_nb = (
     preprocessing.StandardScaler() |
         naive_bayes.GaussianNB()
     )
 


model = (
     preprocessing.StandardScaler() |
     VotingClassifier1([
         linear_model.LogisticRegression(),
         tree.HoeffdingTreeClassifier(),
         naive_bayes.GaussianNB()
     ])
 )


In [37]:
metric = metrics.F1()
print(evaluate.progressive_val_score(dataset, model_lr, metric))
print(evaluate.progressive_val_score(dataset, model_ht, metric))
print(evaluate.progressive_val_score(dataset, model_nb, metric))
print(evaluate.progressive_val_score(dataset, model, metric))


F1: 0.221093
F1: 0.374481
F1: 0.387815
F1: 0.399286
